# Actividad - Proyecto práctico (Versión GPU Segura)
> **VERSIÓN GPU SEGURA**: Configuración simplificada de GPU que evita errores de configuración mientras mantiene optimizaciones.
*   Alumno 1: Granizo, Mateo
*   Alumno 2: Maiolo, Pablo
*   Alumno 3: Miglino, Diego

## **PARTE 1** - Instalación y requisitos previos

In [ ]:
print("Verificando entorno de ejecución...")
try:
  from google.colab import drive
  IN_COLAB=True
  print("Entorno: Google Colab")
except:
  IN_COLAB=False
  print("Entorno: Local")

## **PARTE 3**. Desarrollo y preguntas

In [ ]:
print("Importando librerías...")
from __future__ import division
import numpy as np
import gym
import os
import json
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, Callback
from rl.core import Processor
print("Librerías importadas.")

In [ ]:
print("Configurando GPU de forma segura...")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configuración simple y segura de GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU configurada exitosamente. GPUs disponibles: {len(gpus)}")
        print(f"GPU principal: {gpus[0]}")
    except RuntimeError as e:
        print(f"Error configurando GPU: {e}")
        print("Continuando con configuración por defecto...")
else:
    print("No se detectaron GPUs - usando CPU")

# Verificar que TensorFlow puede usar GPU
print(f"TensorFlow puede usar GPU: {tf.test.is_gpu_available()}")
print(f"Dispositivos disponibles: {tf.config.list_physical_devices()}")

#### Configuración optimizada

In [ ]:
print("Configurando entorno optimizado...")
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

# Semillas para reproducibilidad
np.random.seed(42)
env.seed(42)
tf.random.set_seed(42)

nb_actions = env.action_space.n
print(f"Número de acciones disponibles: {nb_actions}")
print(f"Espacio de observación: {env.observation_space.shape}")

In [ ]:
print("Definiendo procesador optimizado...")
class OptimizedAtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3
        # Procesamiento eficiente
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        # Normalización eficiente
        processed_batch = batch.astype('float32') / 255.0
        return processed_batch - 0.5

    def process_reward(self, reward):
        # Sin clipping para Space Invaders - mejor rendimiento
        return reward

print("Procesador optimizado definido.")

### 1. Red neuronal optimizada

In [ ]:
print("Construyendo red neuronal optimizada...")
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

if K.image_data_format() == 'channels_last':
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

# Arquitectura optimizada pero estable
model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
# Capa adicional para mejor representación
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Flatten())

# Capas densas optimizadas
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

print(model.summary())
print(f"Parámetros totales: {model.count_params():,}")

### 2. Configuración DQN optimizada

In [ ]:
print("Configurando agente DQN optimizado...")

# Memoria optimizada pero segura
memory = SequentialMemory(limit=750000, window_length=WINDOW_LENGTH)
processor = OptimizedAtariProcessor()

# Política de exploración optimizada
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1.0, value_min=0.1, value_test=0.05,
                              nb_steps=1000000)

# Configuración DQN optimizada
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=50000,
               gamma=0.995,  # Ligeramente mayor para mejor planificación
               train_interval=4, 
               batch_size=32,
               delta_clip=1.0,
               target_model_update=10000,
               enable_double_dqn=True,
               enable_dueling_network=False)

# Optimizador mejorado
optimizer = Adam(learning_rate=2e-4)  # Ligeramente más alto
dqn.compile(optimizer, metrics=['mae'])

print("Agente DQN optimizado configurado.")

### 3. Callbacks optimizados y entrenamiento

In [ ]:
print("Definiendo callbacks optimizados...")

class OptimizedMilestoneRecorder(Callback):
    def __init__(self, env, video_folder='videos/', milestones=[40, 60, 80, 100, 150]):
        super(OptimizedMilestoneRecorder, self).__init__()
        self.env = env
        self.video_folder = video_folder
        self.milestones = sorted(milestones)
        self.achieved_milestones = []
        self.best_reward = -float('inf')
        self.episode_rewards = []
        self.recent_rewards = []
        
        print(f"[Callback] OptimizedMilestoneRecorder inicializado. Hitos: {self.milestones}")
        if not os.path.exists(video_folder):
            print(f"[Callback] Creando carpeta de videos: {video_folder}")
            os.makedirs(video_folder)

    def on_episode_end(self, episode, logs={}):
        reward = logs.get('episode_reward')
        if reward is None:
            return
        
        self.episode_rewards.append(reward)
        self.recent_rewards.append(reward)
        
        # Mantener solo las últimas 100 recompensas
        if len(self.recent_rewards) > 100:
            self.recent_rewards.pop(0)
        
        avg_reward = np.mean(self.recent_rewards)
        
        # Guardar mejor modelo
        if reward > self.best_reward:
            self.best_reward = reward
            best_weights_filename = f'dqn_{env_name}_best_gpu_safe_weights.h5f'
            print(f'\n[Callback] ¡Nuevo mejor resultado! Recompensa: {reward:.2f} (Promedio: {avg_reward:.2f})')
            print(f'[Callback] Guardando mejor modelo: {best_weights_filename}')
            self.model.save_weights(best_weights_filename, overwrite=True)
        
        # Verificar hitos
        for milestone in self.milestones:
            if reward >= milestone and milestone not in self.achieved_milestones:
                print(f'\n[Callback] ¡Hito alcanzado! Recompensa: {reward:.2f} >= {milestone}')
                
                weights_filename = f'dqn_{env_name}_weights_gpu_safe_reward_{milestone}.h5f'
                print(f"[Callback] Guardando pesos del hito: {weights_filename}")
                self.model.save_weights(weights_filename, overwrite=True)
                
                # Intentar grabar video
                self._record_video(milestone)
                self.achieved_milestones.append(milestone)
                break
        
        # Progreso cada 50 episodios
        if episode % 50 == 0 and episode > 0:
            print(f'\n[Progreso] Episodio {episode}: Recompensa actual: {reward:.2f}, '
                  f'Promedio últimos 100: {avg_reward:.2f}, Mejor: {self.best_reward:.2f}')
    
    def _record_video(self, milestone):
        video_path = os.path.join(self.video_folder, f'milestone_gpu_safe_reward_{milestone}')
        print(f"[Callback] Intentando grabar video en: {video_path}")
        try:
            video_env = None
            
            # Intentar diferentes métodos de grabación
            try:
                from gym.wrappers.record_video import RecordVideo
                video_env = RecordVideo(self.env, video_path)
                print(f"[Callback] Usando gym.wrappers.record_video.RecordVideo")
            except ImportError:
                try:
                    video_env = gym.wrappers.RecordVideo(self.env, video_path)
                    print(f"[Callback] Usando gym.wrappers.RecordVideo")
                except AttributeError:
                    try:
                        video_env = gym.wrappers.Monitor(self.env, video_path, force=True)
                        print(f"[Callback] Usando gym.wrappers.Monitor")
                    except Exception:
                        pass
            
            if video_env is not None:
                self.model.test(video_env, nb_episodes=1, visualize=False)
                video_env.close()
                print(f"[Callback] Video guardado exitosamente")
            else:
                print(f"[Callback] No se pudo grabar video, continuando...")
                
        except Exception as e:
            print(f"[Callback] Error al grabar video: {e}")

# Configurar archivos de salida
log_filename = f'dqn_{env_name}_log_gpu_safe.json'
weights_filename = f'dqn_{env_name}_weights_gpu_safe.h5f'

callbacks = [
    FileLogger(log_filename, interval=1000),
    OptimizedMilestoneRecorder(env),
    ModelIntervalCheckpoint(f'dqn_{env_name}_gpu_safe_checkpoint_{{step}}.h5f', interval=250000)
]

print("Callbacks optimizados configurados.")

In [ ]:
print("\n" + "="*60)
print("INICIANDO ENTRENAMIENTO OPTIMIZADO GPU SEGURO")
print("="*60)
print(f"Pasos de entrenamiento: 1,750,000")
print(f"Pasos de calentamiento: 50,000")
print(f"Learning rate: 2e-4")
print(f"Gamma: 0.995")
print(f"Batch size: 32")
print(f"Memoria: 750,000 experiencias")
print(f"Target update: cada 10,000 pasos")
print(f"Sin clipping de recompensas")
print("="*60 + "\n")

# Entrenamiento optimizado
dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, visualize=False)

print("\n" + "="*60)
print("ENTRENAMIENTO COMPLETADO")
print("="*60)

# Guardar modelo final
print(f"Guardando pesos finales en: {weights_filename}")
dqn.save_weights(weights_filename, overwrite=True)
print("Pesos finales guardados.")

### 4. Análisis de resultados

In [ ]:
print("Analizando resultados del entrenamiento...")
try:
    with open(log_filename, 'r') as f:
        log_data = json.load(f)
    
    episode_rewards = log_data.get('episode_reward', [])
    nb_episode_steps = log_data.get('nb_episode_steps', [])
    
    if episode_rewards and nb_episode_steps:
        # Calcular estadísticas
        max_reward = max(episode_rewards)
        mean_reward = np.mean(episode_rewards)
        std_reward = np.std(episode_rewards)
        
        # Promedio móvil
        window_size = 100
        moving_avg = []
        for i in range(len(episode_rewards)):
            start_idx = max(0, i - window_size + 1)
            moving_avg.append(np.mean(episode_rewards[start_idx:i+1]))
        
        print(f"\nEstadísticas del entrenamiento:")
        print(f"Recompensa máxima: {max_reward:.2f}")
        print(f"Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")
        print(f"Número total de episodios: {len(episode_rewards)}")
        print(f"Promedio últimos 100 episodios: {np.mean(episode_rewards[-100:]):.2f}")
        
        # Gráfico principal
        plt.figure(figsize=(15, 8))
        
        steps_cumsum = np.cumsum(nb_episode_steps)
        plt.plot(steps_cumsum, episode_rewards, alpha=0.3, color='lightblue', label='Recompensa por episodio')
        plt.plot(steps_cumsum, moving_avg, color='darkblue', linewidth=2, label=f'Promedio móvil ({window_size} episodios)')
        plt.axhline(y=40, color='orange', linestyle='--', alpha=0.7, label='Objetivo básico (40)')
        plt.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='Objetivo intermedio (80)')
        plt.axhline(y=150, color='green', linestyle='--', alpha=0.7, label='Objetivo avanzado (150)')
        plt.title('Progreso del Entrenamiento DQN GPU Seguro - Space Invaders', fontsize=14, fontweight='bold')
        plt.xlabel('Pasos de Entrenamiento')
        plt.ylabel('Recompensa del Episodio')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Análisis de convergencia
        if len(episode_rewards) > 500:
            first_half = episode_rewards[:len(episode_rewards)//2]
            second_half = episode_rewards[len(episode_rewards)//2:]
            improvement = np.mean(second_half) - np.mean(first_half)
            print(f"\nAnálisis de convergencia:")
            print(f"Promedio primera mitad: {np.mean(first_half):.2f}")
            print(f"Promedio segunda mitad: {np.mean(second_half):.2f}")
            print(f"Mejora: {improvement:.2f} ({improvement/np.mean(first_half)*100:.1f}%)")
    
    else:
        print("No se encontraron datos de recompensa en el archivo de log.")
        
except FileNotFoundError:
    print(f"El archivo de log '{log_filename}' no fue encontrado.")
except Exception as e:
    print(f"Error al procesar el archivo de log: {e}")

### 5. Evaluación final del agente

In [ ]:
print("\n" + "="*60)
print("EVALUACIÓN FINAL DEL AGENTE")
print("="*60)

# Cargar el mejor modelo
best_weights_file = f'dqn_{env_name}_best_gpu_safe_weights.h5f'
if os.path.exists(best_weights_file):
    print(f"Cargando mejor modelo: {best_weights_file}")
    dqn.load_weights(best_weights_file)
else:
    print(f"Cargando modelo final: {weights_filename}")
    dqn.load_weights(weights_filename)

print("\nEvaluando agente con 20 episodios...")
test_results = dqn.test(env, nb_episodes=20, visualize=False)

print(f"\nResultados de la evaluación:")
print(f"Recompensa promedio: {np.mean(test_results.history['episode_reward']):.2f}")
print(f"Recompensa máxima: {np.max(test_results.history['episode_reward']):.2f}")
print(f"Recompensa mínima: {np.min(test_results.history['episode_reward']):.2f}")
print(f"Desviación estándar: {np.std(test_results.history['episode_reward']):.2f}")

# Mostrar distribución de recompensas de evaluación
plt.figure(figsize=(10, 6))
plt.hist(test_results.history['episode_reward'], bins=10, alpha=0.7, color='lightgreen', edgecolor='black')
plt.axvline(np.mean(test_results.history['episode_reward']), color='red', linestyle='--', linewidth=2, 
           label=f'Media: {np.mean(test_results.history["episode_reward"]):.2f}')
plt.title('Distribución de Recompensas en Evaluación Final', fontsize=12, fontweight='bold')
plt.xlabel('Recompensa')
plt.ylabel('Frecuencia')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("EVALUACIÓN COMPLETADA")
print("="*60)